## Optimize Workforce Planning using Linear Programming with Python
What is the minimum number of temporary workers you need to hire to absorb your weekly workload while ensuring employees retention?


A major challenge faced by Distribution Center (DC) managers is the fluctuation of the workload during the week.
Considering that the average productivity of your workers is stable, the only solution is to adapt your resources to meet the demand of each day.


# I. Workforce Planning: Problem Statement
### 1. Scenario
You are an Inbound Manager of a Distribution Center operated by a Third Party Logistics Company (3PL) for a large retailer.
Your team responsibilities include
Unload Pallets from the Trucks
Scan each pallet and record the received quantity in your Warehouse Management System (WMS)
Put away these pallets to Stock Area
The team’s global productivity is measured each week in (Pallets/Hour). At the beginning of each month, your colleagues from the transportation team share a forecast of the number of pallets to be received every day for the next 4 weeks.

For more flexibility, you will use 100% of temporary workers to build your team.
### Constraint 1: The Supply must meet the demand
If you need 31 workers Monday, you need to secure at least 31 workers for Monday.
### Constraint 2: Minimum working time by worker
To ensure employees retention, you need to guarantee a minimum of 5 consecutive working days per week. Workforce sourcing can be challenging especially if your DC is surrounded by e-commerce fulfilment centres.
Therefore you need to ensure minimum working time for your temporary workers to be an attractive employer.
### Constraint 3: Maximum working time by week
Following the local regulations, each worker needs to rest 2 days after 5 consecutive working days.


# Objective : Minimize the number of workers hired


## Linear Programming Problem
We define a Linear Programming Problem by finding the
the optimal value of a linear function (objective function) of several variables (x[i]), subject to the conditions that the variables are non-negative and satisfy a set of linear inequalities (called linear constraints).
Our problem fits perfectly!

# Variables
x[i]: number of temporary workers hired for shift i
Constraints
- For each day the total number of workers on-duty must be higher than the demand
- Each worker needs to work a minimum of 5 consecutive days per week
- Each worker needs to have 2 days off after 5 consecutive days of work
Objective functions
The total number of temporary workers hired for all shifts i = 1 … 7 should be minimal

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pulp import *
from itertools import chain, repeat


In [11]:
def ncycles(iterable, n):
    "Returns the sequence elements n times"
    return chain.from_iterable(repeat(tuple(iterable), n))

In [31]:
#network parameters
#staff needed for a day
n_staff = [31,45,40,40,48,30,20]

# Days of the week
jours = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [32]:
# Create circular list of days
n_days = [i for i in range(7)]
n_days_c = list(ncycles(n_days, 3)) 


In [33]:
n_days_c

[0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6]

In [34]:
# Working days
list_in = [[n_days_c[j] for j in range(i , i + 5)] for i in n_days_c]

In [35]:
# Workers off by shift for each day
list_excl = [[n_days_c[j] for j in range(i + 1, i + 3)] for i in n_days_c]



In [36]:
#initialise the model
prob = LpProblem("Workforce scheduling", LpMinimize)

#create Variables
start_jours = ['Shift: ' + i for i in jours]
x = LpVariable.dicts('shift_',n_days, lowBound=0, cat='Integer')

#define Objective function
prob += lpSum([x[i] for i in n_days])

#add constraints
# Add constraints
for d, l_excl, staff in zip(n_days, list_excl, n_staff):
    prob += lpSum([x[i] for i in n_days if i not in l_excl]) >= staff


/opt/anaconda3/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [37]:
#solve the model
status = prob.solve()

#the status of the solution is printed to the screen

print("Status:", LpStatus[status])

#how many workers are needed per day?
dct_work = {}

for v in prob.variables():
    dct_work[int(v.name[-1])] = int(v.varValue)
    
    
    
#show Detailed Sizing per Day
dict_sch = {}

for day in dct_work.keys():
    dict_sch[day] = [dct_work[day] if i in list_in[day] else 0 for i in n_days]
df_sch = pd.DataFrame(dict_sch).T
df_sch.columns = jours
df_sch.index = start_jours



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/wt/9gf59rdd2j16lhgptjnc2cxh0000gp/T/1221ef82583f4c578187bb5646af4e10-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/wt/9gf59rdd2j16lhgptjnc2cxh0000gp/T/1221ef82583f4c578187bb5646af4e10-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 69 RHS
At line 77 BOUNDS
At line 85 ENDATA
Problem MODEL has 7 rows, 7 columns and 35 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 51 - 0.00 seconds
Cgl0003I 0 fixed, 7 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 7 rows, 7 columns (7 integer (0 of which binary)) and 35 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 51 found by greedy cover aft

In [38]:
df_sch

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Shift: Monday,17,17,17,17,17,0,0
Shift: Tuesday,0,14,14,14,14,14,0
Shift: Wednesday,0,0,6,6,6,6,6
Shift: Thursday,0,0,0,0,0,0,0
Shift: Friday,11,11,0,0,11,11,11
Shift: Saturday,0,0,0,0,0,0,0
Shift: Sunday,3,3,3,3,0,0,3


In [39]:
# The optimized objective function value is printed to the screen
print("Total number of Staff = ", pulp.value(prob.objective))

Total number of Staff =  51.0
